In [1]:
from cryptoadvance.specter.rpc import BitcoinRPC, RpcError
import os
from binascii import unhexlify

In [2]:
rpc_user = os.getenv('BTC_RPC_USER')
rpc_password = os.getenv('BTC_RPC_PASSWORD')
rpc_host = os.getenv('BTC_RPC_HOST')
rpc_port = os.getenv('BTC_RPC_PORT')

In [3]:
cli = BitcoinRPC(rpc_user, rpc_password, rpc_host, rpc_port)

## (1) getrawtransaction

In [ ]:
raw_tx = cli.getrawtransaction("54e48e5f5c656b26c3bca14a8c95aa583d07ebe84dde3b7dd4a78f4e4186e713", False, "00000000000000ecbbff6bafb7efa2f7df05b227d5c73dca8f2635af32a2e949")

In [ ]:
# Splitting by the amount which is 1 sat for every output (except for the last output which was used as change)
# Reminder: Amount is in little-endian
# This already cuts off the very last output (change address), since this has an amount other than 1 sat
outputs = raw_tx.split("0100000000000000")
len(outputs)
# 948 = 1 input and 947 outputs, the last output won't be used, though

In [ ]:
# based on https://bitcoin.stackexchange.com/questions/35959/how-is-the-whitepaper-decoded-from-the-blockchain-tx-with-1000x-m-of-n-multisi
pdf = ""
for output in outputs[1:-2]:
    # There are 3 65-byte data parts in every output element, preceded with 41 (65 in decimal)
    # At the beginning of each output there are two more bytes (c9 which means 201 bytes in pubkey script follow and 51 which is OP_1) to cut, thus 3 bytes in total
    cur = 6
    pdf += output[cur:cur+130]
    cur += 132
    pdf += output[cur:cur+130]
    cur += 132
    pdf += output[cur:cur+130]

# Same cut with the second last output as above, 3 bytes at the beginning and 2 bytes and the end 
pdf += outputs[-2][6:-4]


with open("bitcoin.pdf", "wb") as f:
    f.write(unhexlify(pdf[16:-16]))

# sha256 hash of the pdf should be b1674191a88ec5cdd733e4240a81803105dc412d6c6708d53ab94fc248f4f553

## (2) gettxout

### a) Calling gettxout one by one for every output

In [6]:
outputs_prun = []
for i in range(0,946):
    output = cli.gettxout("54e48e5f5c656b26c3bca14a8c95aa583d07ebe84dde3b7dd4a78f4e4186e713", i)
    outputs_prun.append(output)

In [ ]:
pdf_prun = ""
for output in outputs_prun[:-1]:
    cur = 4
    pdf_prun += output['scriptPubKey']['hex'][cur:cur+130]
    cur += 132
    pdf_prun += output['scriptPubKey']['hex'][cur:cur+130]
    cur += 132
    pdf_prun += output['scriptPubKey']['hex'][cur:cur+130]

# Same cut with the last output as above
pdf_prun += outputs_prun[-1]['scriptPubKey']['hex'][4:-4]

with open("bitcoin_prun.pdf", "wb") as f:
    f.write(unhexlify(pdf_prun[16:-16]))

# sha256 hash of the pdf should be b1674191a88ec5cdd733e4240a81803105dc412d6c6708d53ab94fc248f4f553

### b) Making only one big RPC call for all outputs

In [ ]:
outputs_prun_2 = cli.multi([("gettxout","54e48e5f5c656b26c3bca14a8c95aa583d07ebe84dde3b7dd4a78f4e4186e713", i) for i in range(0,946)])

In [14]:
pdf_prun_2 = ""
for output in outputs_prun_2[:-1]:
    cur = 4
    pdf_prun_2 += output['result']['scriptPubKey']['hex'][cur:cur+130]
    cur += 132
    pdf_prun_2 += output['result']['scriptPubKey']['hex'][cur:cur+130]
    cur += 132
    pdf_prun_2 += output['result']['scriptPubKey']['hex'][cur:cur+130]

# Same cut with the last output as above
pdf_prun_2 += outputs_prun_2[-1]['result']['scriptPubKey']['hex'][4:-4]

with open("bitcoin_prun_2.pdf", "wb") as f:
    f.write(unhexlify(pdf_prun_2[16:-16]))

# sha256 hash of the pdf should be b1674191a88ec5cdd733e4240a81803105dc412d6c6708d53ab94fc248f4f553

## (3) Unused last output

In [4]:
unused = cli.gettxout("54e48e5f5c656b26c3bca14a8c95aa583d07ebe84dde3b7dd4a78f4e4186e713", 946)

In [5]:
unused

{'bestblock': '00000000000000000001d68c1b02770534c6087c5afd0e138df163bbd285e1a4',
 'confirmations': 438216,
 'value': 1e-08,
 'scriptPubKey': {'asm': 'OP_DUP OP_HASH160 62e907b15cbf27d5425399ebf6f0fb50ebb88f18 OP_EQUALVERIFY OP_CHECKSIG',
  'hex': '76a91462e907b15cbf27d5425399ebf6f0fb50ebb88f1888ac',
  'reqSigs': 1,
  'type': 'pubkeyhash',
  'addresses': ['1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa']},
 'coinbase': False}

In [ ]:
# No sensible message

### File sigs

In [6]:
# Hex signature at the beginning: 25 50 44 46 --> %PDF

In [ ]:
# Trailer: 0A 25 25 45 4F 46 0A --> .%%EOF.

## (4) Comparison with Musk tweet

In [9]:
musk_coinbase = cli.getrawtransaction("88349380e1f6be0eaefcad8f5abe3303769596caeed63d5de6150bf420e8b0f0")
# Different tx then below, this is from a coinbase tx (from the input script)
# Basically the same approach as with the Genesis block

In [10]:
musk_coinbase

'0200000000010151bd3caba9b8cb6183ad6316474cfda67b98ff26f7d213fb909be217766fbc0500000000171600147e6c09c1dd3ec20b9eddc9a4163edb9d0adb7b52fdffffff02a861000000000000160014317f6d807d9e5d3ec1c4ae793a3ed588ad9c5bb50000000000000000226a20496e20726574726f73706563742c2069742077617320696e6576697461626c650247304402201a5fbec6e9d5c50399ddce2a4a18871ddd01da321d8cba08821d3817abdfc7fd022054b0e558e729a1fc3882898b9befb9f979c07f00bd57527acd7e2f441b4fdd7b0121034bc0c94ffba311a4a2864bc14a388b10a402ba825f9e41da80ad0b2deab6117300000000'

In [16]:
musk_opreturn = cli.gettxout("c3a369ccb3667503280b41494b88215f1953268140ac430288be0fc692c9cc58", 1)

In [18]:
musk_opreturn

{'bestblock': '00000000000000000000edb439c8c28b5e77208ded8062df2990032b6915216d',
 'confirmations': 0,
 'value': 0.0,
 'scriptPubKey': {'asm': 'OP_RETURN 496e20726574726f73706563742c2069742077617320696e6576697461626c65',
  'hex': '6a20496e20726574726f73706563742c2069742077617320696e6576697461626c65',
  'type': 'nulldata'},
 'coinbase': False}

In [19]:
# 496e20726574726f73706563742c2069742077617320696e6576697461626c65 in hex  
# In retrospect, it was inevitable in ASCII